In [1]:
import numpy as np
import pydicom
import nibabel as nib
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.metrics import roc_curve, auc
from time import perf_counter
from scipy.stats import wilcoxon
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


In [ ]:
import sklearn
print(sklearn.__version__)

In [20]:
def readDcmPet(pathToDcmFolder):

    locs=[]
    for i in range(len(os.listdir(pathToDcmFolder))):
        file='{}/{}'.format(pathToDcmFolder,os.listdir(pathToDcmFolder)[i])
        ds=pydicom.dcmread(file,force=True)
        if (0x0020,0x1041) in ds:
            locs.append(ds.SliceLocation)
            i0=ds.pixel_array.shape[0]
            i1=ds.pixel_array.shape[1]
    uniLocs=np.sort(np.unique(np.array(locs)))
    img3d=np.zeros((i0,i1,len(uniLocs)))
    for i in range(len(os.listdir(pathToDcmFolder))):
        file='{}/{}'.format(pathToDcmFolder,os.listdir(pathToDcmFolder)[i])
        ds=pydicom.dcmread(file,force=True)
        if (0x0020,0x1041) in ds:
            for j in range(len(uniLocs)):
                if uniLocs[j]==ds.SliceLocation:
                    if (0x0028, 0x1053) in ds and (0x0028, 0x1052) in ds:
                        img3d[:,:,j]=ds.pixel_array*ds[0x0028, 0x1053].value+ds[0x0028, 0x1052].value
                    else:
                        img3d[:,:,j]=ds.pixel_array
    return(img3d)


def preProcess(img,img_height):

    img=cv2.resize(img,(img_height,img_height))
    img=(img-np.min(img))/(np.max(img)-np.min(img))
    return(img.astype('float32'))

def loadImgSlices(img_height):

    df1=pd.read_excel('D:/rpe/DL-RPE-potilaat.xlsx')
    negSlices=[]
    rpeSlices=[]
    rpeIndexes=[]
    caseNames0=[]
    caseNames1=[]
    for i in range(len(df1['caseID'])):
        pathToDcmFolder='D:/rpe/anon_dixon/{}'.format(df1['caseID'][i])
        img3d=readDcmPet(pathToDcmFolder)
        slices=[]
        for j in range(img3d.shape[2]):
            slices.append(preProcess(img3d[:,:,j],img_height))
        if df1['rpe'][i]==0:
            negSlices.append(slices)
            caseNames0.append(df1['caseID'][i])
        else:
            rpeSlices.append(slices)
            indexes=np.zeros((img3d.shape[2]))
            indexes[int(df1['start_slice'][i]):(int(df1['end_slice'][i])+1)]=1
            rpeIndexes.append(indexes)
            caseNames1.append(df1['caseID'][i])
    return(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1)

def divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k):

    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    trainIndexes=[]
    testIndexes=[]
    testCases=[]

    for i in range(len(rpeSlices)):
        if i%5==k:
            for j in range(len(rpeSlices[i])):
                x_test.append(rpeSlices[i][j])
                y_test.append(rpeIndexes[i][j])
                testIndexes.append(j)
            testCases.append(caseNames1[i])
            if i<len(negSlices):
                for j in range(len(negSlices[i])):
                    x_test.append(negSlices[i][j])
                    y_test.append(0)
                    testIndexes.append(j) 
                testCases.append(caseNames0[i])   
        else:
            for j in range(len(rpeSlices[i])):
                x_train.append(rpeSlices[i][j])
                y_train.append(rpeIndexes[i][j])
                trainIndexes.append(j)
            if i<len(negSlices):
                for j in range(len(negSlices[i])):
                    x_train.append(negSlices[i][j])
                    y_train.append(0)  
                    trainIndexes.append(j)

    x_train=np.array(x_train)
    y_train=np.array(y_train)    
    x_test=np.array(x_test)
    y_test=np.array(y_test)
    trainIndexes=np.array(trainIndexes)
    testIndexes=np.array(testIndexes)
    return(x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases)

def augment(x_train,y_train,type):

    x_train1=[]
    y_train1=[]
    for i in range(x_train.shape[0]):
        x_train1.append(x_train[i])
        y_train1.append(y_train[i])
    for i in range(x_train.shape[0]):
        if type=='flip':
            x_train1.append(cv2.flip(x_train[i],1))
        if type=='rotate90':
            x_train1.append(cv2.rotate(x_train[i],cv2.ROTATE_90_CLOCKWISE))
        if type=='blur':
            x_train1.append(cv2.filter2D(x_train[i],-1,np.ones((3,3))/25))
        y_train1.append(y_train[i])
    x_train1=np.array(x_train1)
    y_train1=np.array(y_train1)
    return(x_train1,y_train1)

def predict(x_train,y_train,x_test,numberOfEpochs):
    
    img_height=x_train[0].shape[0]

    model = keras.Sequential([keras.layers.Conv2D(16, 3, activation='relu', input_shape=(img_height,img_height,1)),
                        keras.layers.Conv2D(16, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(32, 3, activation='relu'),
                        keras.layers.Conv2D(32, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(64, 3, activation='relu'),
                        keras.layers.Conv2D(64, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(128, 3, activation='relu'),
                        keras.layers.Conv2D(128, 3, activation='relu'),
                        keras.layers.MaxPooling2D(strides=(2, 2)),
                        keras.layers.Flatten(),
                        keras.layers.Dense(128, activation='relu'),
                        keras.layers.Dense(64, activation='relu'),
                        keras.layers.Dense(32, activation='relu'),
                        keras.layers.Dense(1, activation='sigmoid')
    ]
    )

    model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
        mode='min', patience=5, restore_best_weights=True)

    history=model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,validation_split=0.3,callbacks=[earlystopping],shuffle=True)
    predictions=model.predict(x_test)
    trainPreds=model.predict(x_train)
    plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')

    return([predictions,trainPreds])


def predictInceptionV3(x_train,y_train,x_test,numberOfEpochs):

    img_height=x_train[0].shape[0]

    x_train1=[]
    for i in range(x_train.shape[0]):
        x_train1.append(np.rollaxis(np.array([x_train[i],x_train[i],x_train[i]]),0,3))
    x_train1=np.array(x_train1)
    
    x_test1=[]
    for i in range(x_test.shape[0]):
        x_test1.append(np.rollaxis(np.array([x_test[i],x_test[i],x_test[i]]),0,3))
    x_test1=np.array(x_test1)

    ntf_model = keras.applications.InceptionV3(weights=None,input_shape=(img_height,img_height,3),include_top=False)

    ntf_model.trainable = True
    inputs = keras.Input(shape=(img_height,img_height,3))
    x = ntf_model(inputs, training=True)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)

    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[keras.metrics.BinaryAccuracy()])
    
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
        mode='min', patience=5, restore_best_weights=True)
    
    history=model.fit(x=x_train1, y=y_train, epochs=numberOfEpochs, validation_split=0.3, callbacks=[earlystopping], shuffle=True)
    predictions=model.predict(x_test1)
    trainPreds=model.predict(x_train1)
    plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')

    return([predictions,trainPreds])

def evaluatePreds(predictions,y_test,trainPreds,y_train):

    default=0.5
    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]
    rocdists = (1-tpr)**2+fpr**2
    minRocDist = thresholds[np.argmin(rocdists)]
    equ = thresholds[np.argmin((tpr-1+fpr)**2)]
    threshold=youden

    TN = 0
    FN = 0
    TP = 0
    FP = 0

    for i in range(len(y_test)):
        if predictions[i]<threshold:
            if y_test[i]==0:
                TN+=1
            else:
                FN+=1
        else:
            if y_test[i]==1:
                TP+=1
            else:
                FP+=1

    acc = (TN+TP)/(TN+FN+TP+FP)
    sen = TP/(FN+TP)
    spe = TN/(TN+FP)
    fpr, tpr, thresholds = roc_curve(y_test, predictions, drop_intermediate=False)
    auc1=auc(fpr, tpr)

    return([acc,sen,spe,auc1])

def convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions):

    x_train_vectors=[]
    y_train_vectors=[]
    x_test_vectors=[]
    y_test_vectors=[]

    vector=[]
    y_vector=[]
    for i in range(len(trainIndexes)):
        vector.append(trainPreds[i])
        y_vector.append(y_train[i])
        if i==len(trainIndexes)-1:
            x_train_vectors.append(vector)
            y_train_vectors.append(y_vector)
        else:
            if trainIndexes[i+1]==0:
                x_train_vectors.append(vector)
                y_train_vectors.append(y_vector)
                vector=[]
                y_vector=[]

    vector=[]
    y_vector=[]
    for i in range(len(testIndexes)):
        vector.append(predictions[i])
        y_vector.append(y_test[i])
        if i==len(testIndexes)-1:
            x_test_vectors.append(vector)
            y_test_vectors.append(y_vector)
        else:
            if testIndexes[i+1]==0:
                x_test_vectors.append(vector)
                y_test_vectors.append(y_vector)
                vector=[]
                y_vector=[]

    return(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors)

def evaluateVectors(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors):

    thresholds=[]
    indexes=[]
    for i in range(len(x_train_vectors)):
        means=[]
        for j in range(len(x_train_vectors[i])-4):
            means.append(np.mean([x_train_vectors[i][j],x_train_vectors[i][j+1],x_train_vectors[i][j+2],x_train_vectors[i][j+3],x_train_vectors[i][j+4]]))
        thresholds.append(max(means))
        indexes.append(max(y_train_vectors[i]))
    thresholds=np.array(thresholds)
    indexes=np.array(indexes)
    thresholds0=thresholds[indexes==0]
    thresholds1=thresholds[indexes==1]

    thresholds=np.unique(thresholds)
    fpr=[]
    tpr=[]
    for i in range(len(thresholds)):
        tpr.append((thresholds1[thresholds1>=thresholds[i]]).shape[0]/thresholds1.shape[0])
        fpr.append((thresholds0[thresholds0>=thresholds[i]]).shape[0]/thresholds0.shape[0])
    tpr=np.array(tpr)
    fpr=np.array(fpr)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]

    predictions1=[]
    indexes1=[]
    for i in range(len(x_train_vectors)):
        means=[]
        for j in range(len(x_train_vectors[i])-4):
            means.append(np.mean([x_train_vectors[i][j],x_train_vectors[i][j+1],x_train_vectors[i][j+2],x_train_vectors[i][j+3],x_train_vectors[i][j+4]]))
        if max(means)>=youden:
            predictions1+=x_train_vectors[i]
            indexes1+=y_train_vectors[i]
    predictions1=np.array(predictions1)
    indexes1=np.array(indexes1)
    fpr, tpr, thresholds = roc_curve(indexes1, predictions1, drop_intermediate=False)
    J_stats = tpr - fpr
    youden1 = thresholds[np.argmax(J_stats)]

    TN = 0
    FN = 0
    TP = 0
    FP = 0

    TN_1 = 0
    FN_1 = 0
    TP_1 = 0
    FP_1 = 0

    thresholds=[]
    indexes=[]
    predictions1=[]
    indexes1=[]
    for i in range(len(x_test_vectors)):
        means=[]
        for j in range(len(x_test_vectors[i])-4):
            means.append(np.mean([x_test_vectors[i][j],x_test_vectors[i][j+1],x_test_vectors[i][j+2],x_test_vectors[i][j+3],x_test_vectors[i][j+4]]))
        thresholds.append(max(means))
        indexes.append(max(y_test_vectors[i]))
        if max(means)<youden:
            if indexes[i]==0:
                TN+=1
                TN_1+=len(y_test_vectors[i])
            else:
                FN+=1
                FN_1+=np.sum(y_test_vectors[i])
                TN_1+=len(y_test_vectors[i])-np.sum(y_test_vectors[i])
            predictions1+=list(np.zeros(len(y_test_vectors[i])))
            indexes1+=y_test_vectors[i]
        else:
            if indexes[i]==1:
                TP+=1
            else:
                FP+=1
            for j in range(len(x_test_vectors[i])):
                if x_test_vectors[i][j]<youden1:
                    if y_test_vectors[i][j]==0:
                        TN_1+=1
                    else:
                        FN_1+=1
                else:
                    if y_test_vectors[i][j]==1:
                        TP_1+=1
                    else:
                        FP_1+=1
            predictions1+=x_test_vectors[i]
            indexes1+=y_test_vectors[i]
    thresholds=np.array(thresholds)
    indexes=np.array(indexes)
    thresholds0=thresholds[indexes==0]
    thresholds1=thresholds[indexes==1]
    thresholds=np.unique(thresholds)
    fpr=[]
    tpr=[]
    for i in range(len(thresholds)):
        tpr.append((thresholds1[thresholds1>=thresholds[i]]).shape[0]/thresholds1.shape[0])
        fpr.append((thresholds0[thresholds0>=thresholds[i]]).shape[0]/thresholds0.shape[0])
    tpr=np.array(tpr)
    fpr=np.array(fpr)
    auc1_1=auc(fpr, tpr)
    predictions1=np.array(predictions1)
    indexes1=np.array(indexes1)
    fpr, tpr, thresholds = roc_curve(indexes1, predictions1, drop_intermediate=False)
    auc1=auc(fpr, tpr)

    acc = (TN+TP)/(TN+FN+TP+FP)
    sen = TP/(FN+TP)
    spe = TN/(TN+FP)
    acc_1 = (TN_1+TP_1)/(TN_1+FN_1+TP_1+FP_1)
    sen_1 = TP_1/(FN_1+TP_1)
    spe_1 = TN_1/(TN_1+FP_1)
    return([acc,sen,spe,auc1,acc_1,sen_1,spe_1,auc1_1])

def listFinalIndexes(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors,testCases):

    thresholds=[]
    indexes=[]
    for i in range(len(x_train_vectors)):
        means=[]
        for j in range(len(x_train_vectors[i])-4):
            means.append(np.mean([x_train_vectors[i][j],x_train_vectors[i][j+1],x_train_vectors[i][j+2],x_train_vectors[i][j+3],x_train_vectors[i][j+4]]))
        thresholds.append(max(means))
        indexes.append(max(y_train_vectors[i]))
    thresholds=np.array(thresholds)
    indexes=np.array(indexes)
    thresholds0=thresholds[indexes==0]
    thresholds1=thresholds[indexes==1]

    thresholds=np.unique(thresholds)
    fpr=[]
    tpr=[]
    for i in range(len(thresholds)):
        tpr.append((thresholds1[thresholds1>=thresholds[i]]).shape[0]/thresholds1.shape[0])
        fpr.append((thresholds0[thresholds0>=thresholds[i]]).shape[0]/thresholds0.shape[0])
    tpr=np.array(tpr)
    fpr=np.array(fpr)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]

    predictions1=[]
    indexes1=[]
    for i in range(len(x_train_vectors)):
        means=[]
        for j in range(len(x_train_vectors[i])-4):
            means.append(np.mean([x_train_vectors[i][j],x_train_vectors[i][j+1],x_train_vectors[i][j+2],x_train_vectors[i][j+3],x_train_vectors[i][j+4]]))
        if max(means)>=youden:
            predictions1+=x_train_vectors[i]
            indexes1+=y_train_vectors[i]
    predictions1=np.array(predictions1)
    indexes1=np.array(indexes1)
    fpr, tpr, thresholds = roc_curve(indexes1, predictions1, drop_intermediate=False)
    J_stats = tpr - fpr
    youden1 = thresholds[np.argmax(J_stats)]

    TN = 0
    FN = 0
    TP = 0
    FP = 0

    TN_1 = 0
    FN_1 = 0
    TP_1 = 0
    FP_1 = 0

    thresholds=[]
    indexes=[]
    predictions1=[]
    indexes1=[]
    TN_indexes=[]
    TP_indexes=[]
    FN_indexes=[]
    FP_indexes=[]
    FN_patients=[]
    FP_patients=[]
    for i in range(len(x_test_vectors)):
        means=[]
        for j in range(len(x_test_vectors[i])-4):
            means.append(np.mean([x_test_vectors[i][j],x_test_vectors[i][j+1],x_test_vectors[i][j+2],x_test_vectors[i][j+3],x_test_vectors[i][j+4]]))
        thresholds.append(max(means))
        indexes.append(max(y_test_vectors[i]))
        if max(means)<youden:
            if indexes[i]==0:
                TN+=1
                TN_1+=len(y_test_vectors[i])
                TN_indexes.append('{}_all'.format(i))
            else:
                FN+=1
                FN_1+=np.sum(y_test_vectors[i])
                TN_1+=len(y_test_vectors[i])-np.sum(y_test_vectors[i])
                FN_indexes.append('{}_allP'.format(i))
                TN_indexes.append('{}_allN'.format(i))
                FN_patients.append(testCases[i])
            predictions1+=list(np.zeros(len(y_test_vectors[i])))
            indexes1+=y_test_vectors[i]
        else:
            if indexes[i]==1:
                TP+=1
            else:
                FP+=1
                FP_patients.append(testCases[i])
            for j in range(len(x_test_vectors[i])):
                if x_test_vectors[i][j]<youden1:
                    if y_test_vectors[i][j]==0:
                        TN_1+=1
                        TN_indexes.append('{}_{}'.format(i,j))
                    else:
                        FN_1+=1
                        FN_indexes.append('{}_{}'.format(i,j))
                else:
                    if y_test_vectors[i][j]==1:
                        TP_1+=1
                        TP_indexes.append('{}_{}'.format(i,j))
                    else:
                        FP_1+=1
                        FP_indexes.append('{}_{}'.format(i,j))
            predictions1+=x_test_vectors[i]
            indexes1+=y_test_vectors[i]

    return(TN_indexes,TP_indexes,FN_indexes,FP_indexes,FN_patients,FP_patients)

def infoForROC(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors):

    thresholds=[]
    indexes=[]
    for i in range(len(x_train_vectors)):
        means=[]
        for j in range(len(x_train_vectors[i])-4):
            means.append(np.mean([x_train_vectors[i][j],x_train_vectors[i][j+1],x_train_vectors[i][j+2],x_train_vectors[i][j+3],x_train_vectors[i][j+4]]))
        thresholds.append(max(means))
        indexes.append(max(y_train_vectors[i]))
    thresholds=np.array(thresholds)
    indexes=np.array(indexes)
    thresholds0=thresholds[indexes==0]
    thresholds1=thresholds[indexes==1]

    thresholds=np.unique(thresholds)
    fpr=[]
    tpr=[]
    for i in range(len(thresholds)):
        tpr.append((thresholds1[thresholds1>=thresholds[i]]).shape[0]/thresholds1.shape[0])
        fpr.append((thresholds0[thresholds0>=thresholds[i]]).shape[0]/thresholds0.shape[0])
    tpr=np.array(tpr)
    fpr=np.array(fpr)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]

    predictions1=[]
    indexes1=[]
    for i in range(len(x_train_vectors)):
        means=[]
        for j in range(len(x_train_vectors[i])-4):
            means.append(np.mean([x_train_vectors[i][j],x_train_vectors[i][j+1],x_train_vectors[i][j+2],x_train_vectors[i][j+3],x_train_vectors[i][j+4]]))
        if max(means)>=youden:
            predictions1+=x_train_vectors[i]
            indexes1+=y_train_vectors[i]
    predictions1=np.array(predictions1)
    indexes1=np.array(indexes1)
    fpr, tpr, thresholds = roc_curve(indexes1, predictions1, drop_intermediate=False)
    J_stats = tpr - fpr
    youden1 = thresholds[np.argmax(J_stats)]

    TN = 0
    FN = 0
    TP = 0
    FP = 0

    TN_1 = 0
    FN_1 = 0
    TP_1 = 0
    FP_1 = 0

    thresholds=[]
    indexes=[]
    predictions1=[]
    indexes1=[]
    for i in range(len(x_test_vectors)):
        means=[]
        for j in range(len(x_test_vectors[i])-4):
            means.append(np.mean([x_test_vectors[i][j],x_test_vectors[i][j+1],x_test_vectors[i][j+2],x_test_vectors[i][j+3],x_test_vectors[i][j+4]]))
        thresholds.append(max(means))
        indexes.append(max(y_test_vectors[i]))
        if max(means)<youden:
            if indexes[i]==0:
                TN+=1
                TN_1+=len(y_test_vectors[i])
            else:
                FN+=1
                FN_1+=np.sum(y_test_vectors[i])
                TN_1+=len(y_test_vectors[i])-np.sum(y_test_vectors[i])
            predictions1+=list(np.zeros(len(y_test_vectors[i])))
            indexes1+=y_test_vectors[i]
        else:
            if indexes[i]==1:
                TP+=1
            else:
                FP+=1
            for j in range(len(x_test_vectors[i])):
                if x_test_vectors[i][j]<youden1:
                    if y_test_vectors[i][j]==0:
                        TN_1+=1
                    else:
                        FN_1+=1
                else:
                    if y_test_vectors[i][j]==1:
                        TP_1+=1
                    else:
                        FP_1+=1
            predictions1+=x_test_vectors[i]
            indexes1+=y_test_vectors[i]
    thresholds=np.array(thresholds)
    indexes=np.array(indexes)
    thresholds0=thresholds[indexes==0]
    thresholds1=thresholds[indexes==1]
    thresholds=np.unique(thresholds)
    fpr=[]
    tpr=[]
    for i in range(len(thresholds)):
        tpr.append((thresholds1[thresholds1>=thresholds[i]]).shape[0]/thresholds1.shape[0])
        fpr.append((thresholds0[thresholds0>=thresholds[i]]).shape[0]/thresholds0.shape[0])
    tpr1=np.array(tpr)
    fpr1=np.array(fpr)
    predictions1=np.array(predictions1)
    indexes1=np.array(indexes1)
    fpr, tpr, thresholds = roc_curve(indexes1, predictions1, drop_intermediate=False)

    return([tpr,fpr,tpr1,fpr1])

def intersection(lst1,lst2):
    lst3=[value for value in lst1 if value in lst2]
    return lst3

def vectorsToSameDimension(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors):

    train_real_lengths=[]
    test_real_lengths=[]
    for i in range(len(x_train_vectors)):
        train_real_lengths.append(len(x_train_vectors[i]))
    for i in range(len(x_test_vectors)):
        test_real_lengths.append(len(x_test_vectors[i]))

    maxLength=max(max(train_real_lengths),max(test_real_lengths))
    for i in range(len(x_train_vectors)):
        if len(x_train_vectors[i])<maxLength:
            x_train_vectors[i]=x_train_vectors[i]+list(np.zeros((int(maxLength-len(x_train_vectors[i])))))
            y_train_vectors[i]=y_train_vectors[i]+list(np.zeros((int(maxLength-len(y_train_vectors[i])))))
    for i in range(len(x_test_vectors)):
        if len(x_test_vectors[i])<maxLength:
            x_test_vectors[i]=x_test_vectors[i]+list(np.zeros((int(maxLength-len(x_test_vectors[i])))))
            y_test_vectors[i]=y_test_vectors[i]+list(np.zeros((int(maxLength-len(y_test_vectors[i])))))

    x_train_vectors=np.array(x_train_vectors)
    y_train_vectors=np.array(y_train_vectors)
    x_test_vectors=np.array(x_test_vectors)
    y_test_vectors=np.array(y_test_vectors)
    train_real_lengths=np.array(train_real_lengths)
    test_real_lengths=np.array(test_real_lengths)

    return(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors,train_real_lengths,test_real_lengths)

def evaluateLSTMvectors(predicted_vectors,y_test,test_real_lengths,trainPred_vectors,y_train,train_real_lengths):

    predictions=[]
    trainPreds=[]
    for i in range(len(predicted_vectors)):
        predictions=predictions+list(predicted_vectors[i][0:test_real_lengths[i]])
    for i in range(len(trainPred_vectors)):
        trainPreds=trainPreds+list(trainPred_vectors[i][0:train_real_lengths[i]])
    predictions=np.array(predictions)
    trainPreds=np.array(trainPreds)
    y_test=y_test[0:len(predictions)]
    y_train=y_train[0:len(trainPreds)]
    acc,sen,spe,auc1=evaluatePreds(predictions,y_test,trainPreds,y_train)
    return([acc,sen,spe,auc1])

In [11]:
pathToDcmFolder='D:/rpe/anon_dixon/case0001'

In [14]:
img3d=readDcmPet(pathToDcmFolder)

In [ ]:
img3d.shape

In [ ]:
plt.imshow(cv2.filter2D(cv2.resize(img3d[:,:,0],(128,128)),-1,np.ones((3,3))/25),cmap='gray')

In [ ]:
df1=pd.read_excel('D:/rpe/DL-RPE-potilaat.xlsx')

In [3]:
img_height=128
negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1=loadImgSlices(img_height)

In [ ]:
print(len(negSlices))
print(len(rpeSlices))
print(len(rpeIndexes))

In [ ]:
j=[]
for i in range(len(rpeIndexes)):
    j.append(len(rpeIndexes[i]))
for i in range(len(negSlices)):
    j.append(len(negSlices[i]))
print(max(j))

In [ ]:
k=0
x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
print(x_train.shape)
print(y_train.shape)
print(np.sum(y_train))
print(x_test.shape)
print(y_test.shape)
print(np.sum(y_test))

In [ ]:
x_train1,y_train1=augment(x_train,y_train,'flip')
print(x_train1.shape)
print(y_train1.shape)

In [75]:
df1=pd.read_excel('D:/rpe/DL-RPE-potilaat.xlsx')
j=[]
j1=[]
j2=[]
j3=[]
for i in range(len(df1['caseID'])):
    pathToDcmFolder='D:/rpe/anon_dixon/{}'.format(df1['caseID'][i])
    file='{}/{}'.format(pathToDcmFolder,os.listdir(pathToDcmFolder)[0])
    ds=pydicom.dcmread(file,force=True)
    j.append(ds.SpacingBetweenSlices)
    j1.append(ds.PixelSpacing[0])
    j2.append(ds.pixel_array.shape[0])
    j3.append(ds.WindowWidth)

In [ ]:
ji=[]
for i in range(len(j1)):
    ji.append(j1[i]*j2[i])
print(ji)

In [ ]:
print(max(ji))

In [ ]:
numberOfEpochs=15
for type in ['flip','rotate90','blur']:
    for k in range(5):
        x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
        x_train,y_train=augment(x_train,y_train,type)
        for j in range(6):
            startTime=perf_counter()
            predictions,trainPreds=predict(x_train,y_train,x_test,numberOfEpochs)
            print(evaluatePreds(predictions,y_test,trainPreds,y_train))
            np.savetxt('rpe_unet_{}_{}_{}_trainPreds.txt'.format(type,k,j),trainPreds)
            np.savetxt('rpe_unet_{}_{}_{}_predictions.txt'.format(type,k,j),predictions)
            endTime=perf_counter()
            processingTime=endTime-startTime
            np.savetxt('rpe_unet_{}_{}_{}_time.txt'.format(type,k,j),np.array([k,j,processingTime]))

In [ ]:
y=[]
for k in range(5):
    x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
    x_train,y_train=augment(x_train,y_train,'flip')
    for j in range(6):
        trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
        predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
        #y.append(evaluatePreds(predictions,y_test,trainPreds,y_train)[3])
        time=np.loadtxt('rpe_unet_flip_{}_{}_time.txt'.format(k,j))[2]
        y.append(time/60)
y=np.array(y)
print(y)

type='blur'
y1=[]
for k in range(5):
    x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
    x_train,y_train=augment(x_train,y_train,type)
    for j in range(6):
        trainPreds=np.loadtxt('rpe_unet_{}_{}_{}_trainPreds.txt'.format(type,k,j))
        predictions=np.loadtxt('rpe_unet_{}_{}_{}_predictions.txt'.format(type,k,j))
        #y1.append(evaluatePreds(predictions,y_test,trainPreds,y_train)[3])
        time=np.loadtxt('rpe_unet_{}_{}_{}_time.txt'.format(type,k,j))[2]
        y1.append(time/60)
y1=np.array(y1)
print(y1)

print(np.mean(y))
print(np.std(y))
print(np.median(y))
print(np.mean(y1))
print(np.std(y1))
print(np.median(y1))
print(wilcoxon(y,y1))

In [ ]:
k=0
j=0
x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
x_train,y_train=augment(x_train,y_train,'flip')
trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
print(evaluatePreds(predictions,y_test,trainPreds,y_train))

In [49]:
x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors=convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions)

In [ ]:
print(evaluateVectors(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors))

In [52]:
x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors,train_real_lengths,test_real_lengths=vectorsToSameDimension(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors)

In [ ]:
print(x_train_vectors.shape)
print(y_train_vectors.shape)
print(x_test_vectors.shape)
print(y_test_vectors.shape)

In [ ]:
train_indexes=[]
test_indexes=[]
for i in range(len(y_train_vectors)):
    train_indexes.append(np.max(y_train_vectors[i]))
for i in range(len(y_test_vectors)):
    test_indexes.append(np.max(y_test_vectors[i]))
train_indexes=np.array(train_indexes)
test_indexes=np.array(test_indexes)
print(train_indexes.shape)
print(test_indexes.shape)

In [ ]:
embedding_vector_length=12

model = keras.Sequential([keras.layers.Embedding(12,embedding_vector_length),
                        keras.layers.LSTM(10),
                        keras.layers.Dense(1, activation='sigmoid')
]
)

model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
)

numberOfEpochs=30
history=model.fit(x=x_train_vectors,y=train_indexes,epochs=numberOfEpochs,validation_split=0.3,shuffle=True)
predicted_indexes=model.predict(x_test_vectors)
trainPred_indexes=model.predict(x_train_vectors)

In [ ]:
rf=RandomForestClassifier()
rf.fit(x_train_vectors,train_indexes)
predicted_indexes=rf.predict(x_test_vectors)
trainPred_indexes=rf.predict(x_train_vectors)

In [47]:
svm1=svm.SVC()
svm1.fit(x_train_vectors,train_indexes)
predicted_indexes=svm1.predict(x_test_vectors)
trainPred_indexes=svm1.predict(x_train_vectors)

In [ ]:
print(evaluatePreds(predicted_indexes,test_indexes,trainPred_indexes,train_indexes))

In [ ]:
y=[]
y1=[]
for k in range(5):
    x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
    for j in range(6):
        trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
        predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
        x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors=convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions)
        y.append(evaluateVectors(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors)[3])
        x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors,train_real_lengths,test_real_lengths=vectorsToSameDimension(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors)
        train_indexes=[]
        test_indexes=[]
        for l in range(len(y_train_vectors)):
            train_indexes.append(np.max(y_train_vectors[l]))
        for l in range(len(y_test_vectors)):
            test_indexes.append(np.max(y_test_vectors[l]))
        train_indexes=np.array(train_indexes)
        test_indexes=np.array(test_indexes)
        rf=RandomForestClassifier()
        rf.fit(x_train_vectors,train_indexes)
        predicted_indexes=rf.predict(x_test_vectors)
        trainPred_indexes=rf.predict(x_train_vectors)
        #svm1=svm.SVC()
        #svm1.fit(x_train_vectors,train_indexes)
        #predicted_indexes=svm1.predict(x_test_vectors)
        #trainPred_indexes=svm1.predict(x_train_vectors)
        y1.append(evaluatePreds(predicted_indexes,test_indexes,trainPred_indexes,train_indexes)[3])
y=np.array(y)
print(y)
y1=np.array(y1)
print(y1)

print(np.mean(y))
print(np.std(y))
print(np.median(y))
print(np.mean(y1))
print(np.std(y1))
print(np.median(y1))
print(wilcoxon(y,y1))

In [ ]:
y=[]
for k in range(5):
    x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
    for j in range(6):
        trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
        predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
        x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors=convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions)
        y.append(evaluateVectors(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors)[7])
y=np.array(y)
print(y)

print(np.mean(y))
print(np.std(y))
print(np.median(y))

In [34]:
k=4
x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
j=0
trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors=convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions)
TN_indexes1,TP_indexes1,FN_indexes1,FP_indexes1,FN_patients1,FP_patients1=listFinalIndexes(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors,testCases)
for j in range(1,6):
    trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
    predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
    x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors=convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions)
    TN_indexes,TP_indexes,FN_indexes,FP_indexes,FN_patients,FP_patients=listFinalIndexes(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors,testCases)
    TN_indexes1=intersection(TN_indexes1,TN_indexes)
    TP_indexes1=intersection(TP_indexes1,TP_indexes)
    FN_indexes1=intersection(FN_indexes1,FN_indexes)
    FP_indexes1=intersection(FP_indexes1,FP_indexes)
    FN_patients1=intersection(FN_patients1,FN_patients)
    FP_patients1=intersection(FP_patients1,FP_patients)


In [ ]:
print(FN_patients1)

In [ ]:
print(FP_patients1)

In [ ]:
print(TN_indexes1)

In [11]:
vector=[]
x_vectors=[]
for i in range(len(testIndexes)):
    vector.append(x_test[i])
    if i==len(testIndexes)-1:
        x_vectors.append(vector)
    else:
        if testIndexes[i+1]==0:
            x_vectors.append(vector)
            vector=[]

In [ ]:
img=np.ones((260,260))
img[0:128,0:128]=x_vectors[1][15]
img[0:128,132:260]=x_vectors[7][21]
img[132:260,0:128]=x_vectors[3][13]
img[132:260,132:260]=x_vectors[13][10]
plt.imshow(img,cmap='gray')
fig=plt.gcf()
plt.axis('off')
#fig.savefig('fig_TN',bbox_inches='tight')

In [ ]:
plt.imshow(x_test[0],cmap='gray')

In [ ]:
kuv=np.ones((768,128))
for i in range(512):
    for j in range(128):
        kuv[767-i-2*j,j]=x_test[0][127-int(i/4),j]
plt.imshow(kuv,cmap='gray')
fig=plt.gcf()
plt.axis('off')
fig.savefig('fig1',bbox_inches='tight')

In [ ]:
print(x_test_vectors[0][0:7])

In [ ]:
kuv=np.ones((768,768))
for u in range(12):
    u1=[540,510,480,450,420,390,360,120,90,60,30,0][11-u]
    u0=39-u
    if u>4:
        u0=11-u
    for i in range(128):
        for j in range(512):
            kuv[767-i-u1,j+i]=x_test[u0][127-i,int(j/4)]
plt.imshow(kuv,cmap='gray')
fig=plt.gcf()
plt.axis('off')
#fig.savefig('figl',bbox_inches='tight')

In [ ]:
print(TN_indexes)

In [83]:
tprList=[]
fprList=[]

for k in range(5):
    x_train,y_train,x_test,y_test,trainIndexes,testIndexes,testCases=divideIntoSets(negSlices,rpeSlices,rpeIndexes,caseNames0,caseNames1,k)
    for j in range(6):
        trainPreds=np.loadtxt('rpe_unet_flip_{}_{}_trainPreds.txt'.format(k,j))
        predictions=np.loadtxt('rpe_unet_flip_{}_{}_predictions.txt'.format(k,j))
        x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors=convertIntoVectors(y_train,y_test,trainIndexes,testIndexes,trainPreds,predictions)
        tpr,fpr,tpr1,fpr1=infoForROC(x_train_vectors,y_train_vectors,x_test_vectors,y_test_vectors)
        tprList.append(tpr1)
        fprList.append(fpr1)

In [84]:
fpr=0.001*np.array(range(1000))
tprArray=np.zeros((30,1000))
for i in range(30):
    for j in range(1000):
        tprArray[i,j]=np.min(tprList[i][fprList[i]>0.001*j])
tpr=np.median(tprArray,axis=0)

In [ ]:
plt.figure()
plt.plot(fpr,tpr,color='blue')
plt.plot([0,1],[0,1],'--',color='lightgray')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve on patient level')
fig=plt.gcf()
#fig.savefig('figr1',bbox_inches='tight')
plt.show()